### Incremental Data Ingestion with Auto Loader

Spark Structured Streaming is a feature of Apache Spark that enables for simplified configuration when processing incremental datasets. Historically, streaming big data has been driven by the need to reduce latency for the sake of providing near real-time analytics; however, this lesson focuses more on implementing incremental data processing.

While not absolutely required when working with a *data lakehouse*, many enterprise implementations will derive substantial benefit from incremental data processing. Incremental ETL is important since it allows us to deal solely with new data that has been encountered since the last ingestion. Reliably processing only the new data reduces redundant processing and helps enterprises reliably scale data pipelines.  The first step for any successful data lakehouse implementation is ingesting into a Delta Lake table from cloud storage. Historically, ingesting files from a data lake into a database has been a complicated process; however, Databricks Auto Loader provides an easy-to-use mechanism for incrementally and efficiently processing new data files as they arrive in cloud file storage. Due to the benefits and scalability that Auto Loader delivers, Databricks recommends its use as general **best practice** when ingesting data from cloud object storage.

#### Objectives
* Execute Auto Loader code to incrementally ingest data from cloud storage to Delta Lake
* Describe what happens when a new file arrives in a directory configured for Auto Loader
* Query a table fed by a streaming Auto Loader query

#### 1.0. Import Shared Utilities and Data Files
This lab will demonstrate ingesting artificially generated medical data, in JSON format, that simulates heart rate monitor signals captured from numerous devices; therefore, this data represents what would be expected from a *Streaming* data source.

| Field | Type |
| --- | --- 
| device_id | int |
| mrn | long |
| time | double |
| heartrate | double |

First, run the following cell to import the data and make various utilities available for our experimentation.

In [0]:
%run ./Includes/4.0-setup

#### 2.0. Using Auto Loader
In the cell below, a function is defined to demonstrate using Databricks Auto Loader with the PySpark API. This code includes both a Structured Streaming read and write. If you wish to learn more about Auto Loader options, refer to the <a href="https://docs.databricks.com/spark/latest/structured-streaming/auto-loader.html" target="_blank">documentation</a>.

Note that when using Auto Loader with automatic <a href="https://docs.databricks.com/spark/latest/structured-streaming/auto-loader-schema.html" target="_blank">schema inference and evolution</a>, the 4 arguments shown here should allow ingestion of most datasets. These arguments are explained below.

| argument | what it is | how it's used |
| --- | --- | --- |
| **`data_source`** | The directory of the source data | Auto Loader will detect new files as they arrive in this location and queue them for ingestion; passed to the **`.load()`** method |
| **`source_format`** | The format of the source data |  While the format for all Auto Loader queries will be **`cloudFiles`**, the format of the source data should always be specified for the **`cloudFiles.format`** option |
| **`table_name`** | The name of the target table | Spark Structured Streaming supports writing directly to Delta Lake tables by passing a table name as a string to the **`.table()`** method. Note that you can either append to an existing table or create a new table |
| **`checkpoint_directory`** | The location for storing metadata about the stream | This argument is passed to the **`checkpointLocation`** and **`cloudFiles.schemaLocation`** options. Checkpoints keep track of streaming progress, while the schema location tracks updates to the fields in the source dataset |

**NOTE**: The code below has been streamlined to demonstrate Auto Loader functionality. Later we'll see how additional transformations can be applied to source data before saving them to Delta Lake.

In [0]:
def autoload_to_table(data_source, source_format, table_name, checkpoint_directory):
    query = (spark.readStream
                  .format("cloudFiles")
                  .option("cloudFiles.format", source_format)
                  .option("cloudFiles.schemaLocation", checkpoint_directory)
                  .load(data_source)
                  .writeStream
                  .option("checkpointLocation", checkpoint_directory)
                  .option("mergeSchema", "true")
                  .table(table_name))
    return query

We will now use the previously defined function and some path variables defined in the setup script to begin an Auto Loader stream that reads from a source directory of JSON files.

In [0]:
query = autoload_to_table(data_source = f"{DA.paths.working_dir}/tracker",
                          source_format = "json",
                          table_name = "target_table",
                          checkpoint_directory = f"{DA.paths.checkpoints}/target_table")


Because Auto Loader uses Spark Structured Streaming to load data incrementally, the code above doesn't appear to finish executing. That's because this is a **continuously active query**, which means that as soon as new data arrives in our data source, it will be processed through our logic and loaded into our target table.

##### 2.1. Helper Function for Streaming Lessons
This lesson demonstrates combining streaming functions with batch and streaming queries against the results of their operations. This notebook is for instructional purposes and intended for interactive, cell-by-cell execution. That being said, here we define a helper function to prevent our notebook from executing the next cell just long enough to ensure data has been written out by a given streaming query. This code would not be necessary in a production job.

In [0]:
def block_until_stream_is_ready(query, min_batches=2):
    import time
    while len(query.recentProgress) < min_batches:
        time.sleep(5) # Give it a couple of seconds

    print(f"The stream has processed {len(query.recentProgress)} batchs")

block_until_stream_is_ready(query)

#### 3.0. Query Target Table
Once data has been ingested into Delta Lake using the Auto Loader, it can be interacted with in just the same manner as any SQL database table.

In [0]:
%sql
SELECT * FROM target_table

Note that the **`_rescued_data`** column is added by Auto Loader automatically to capture any data that might be malformed and not fit into the table otherwise.  While Auto Loader captured the field names for our data correctly, note that it encoded all fields as **`STRING`** type. Because JSON is a text-based format, this is the safest and most permissive type, ensuring that the least amount of data is dropped or ignored at ingestion due to type mismatch.

In [0]:
%sql
DESCRIBE TABLE target_table

Next, define a temporary view that summarizes the recordings in our target table.  This view will be used to demonstrate how new data is automatically ingested with Auto Loader.

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW device_counts AS
  SELECT device_id, count(*) total_recordings
  FROM target_table
  GROUP BY device_id;
  
SELECT * FROM device_counts

####4.0. Land New Data

As mentioned previously, Auto Loader is configured to incrementally process files from a directory in cloud object storage into a Delta Lake table.  We have configured and are currently executing a query to process JSON files from the location specified by **`source_path`** into a table named **`target_table`**. Let's review the contents of the **`source_path`** directory.

In [0]:
files = dbutils.fs.ls(f"{DA.paths.working_dir}/tracker")
display(files)

At present, you should see a single JSON file listed in this location. The function invoked in the following cell was configured by our setup script to allow us to mimic an external system writing new data to this directory. Each time you execute the cell below, a new file will land in the **`source_path`** directory.

In [0]:
DA.data_factory.load()

When we list the contents of the **`source_path`** again using the cell below we should see an additional JSON file for each time you ran the previous cell.

In [0]:
files = dbutils.fs.ls(f"{DA.paths.working_dir}/tracker")
display(files)

#### 5.0. Tracking Ingestion Progress
Historically, many systems have been configured to either reprocess all records in a source directory to calculate current results or require data engineers to implement custom logic to identify new data that's arrived since the last time a table was updated.  With Auto Loader, your table has already been updated. Run the query below to confirm that new data has been ingested.

In [0]:
%sql
SELECT * FROM device_counts

The Auto Loader query we configured earlier automatically detects and processes records from the source directory into the target table. While there is a slight delay as records are ingested, an Auto Loader query executing with default streaming configuration should update results in near real time.  The query below shows the table history. A new table version should be indicated for each **`STREAMING UPDATE`**. These update events coincide with new batches of data arriving at the source.

In [0]:
%sql
DESCRIBE HISTORY target_table

#### 6.0. Clean Up
Feel free to continue landing new data and exploring the table results with the cells above. When you're finished, run the following cell to stop all active streams and remove created resources.

In [0]:
DA.cleanup()